In [17]:
from dplython import select, DplyFrame, X, arrange, count, sift, head, summarize, group_by, tail, mutate
import numpy as np
import pandas as pd
import string
import rcurl

In [18]:
date_of_study = "04-04-2020"
# Historical data
covid_hist = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-30-2020.csv")

covid_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3439 entries, 0 to 3438
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FIPS            3148 non-null   float64
 1   Admin2          3178 non-null   object 
 2   Province_State  3265 non-null   object 
 3   Country_Region  3439 non-null   object 
 4   Last_Update     3439 non-null   object 
 5   Lat             3439 non-null   float64
 6   Long_           3439 non-null   float64
 7   Confirmed       3439 non-null   int64  
 8   Deaths          3439 non-null   int64  
 9   Recovered       3439 non-null   int64  
 10  Active          3439 non-null   int64  
 11  Combined_Key    3439 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 322.5+ KB


In [19]:
covid_us_hist = covid_hist[(covid_hist['Country_Region'] == "US") & (covid_hist['FIPS'].isnull() == False)]
covid_us_hist.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,3/30/20 22:52,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,3/30/20 22:52,30.295065,-92.414197,11,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,3/30/20 22:52,37.767072,-75.632346,6,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,3/30/20 22:52,43.452658,-116.241552,113,2,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,3/30/20 22:52,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [20]:
# Import outcome data from JHU CSSE
covid = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(date_of_study))

#covid = read.csv(text=getURL(paste0("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/",date_of_study,".csv")))

covid_us = covid[(covid['Country_Region'] == "US") & (covid['FIPS'].isnull() == True)]
covid_us1 = covid_us_hist[(covid_us_hist['FIPS'].isin(covid_us)) & (covid_us_hist['Confirmed'] == 0) & (covid_us_hist['Deaths'] == 0) & (covid_us_hist['FIPS'].isnull() == False)]

covid_us = covid_us.append(covid_us1)

In [21]:
# Import exposure PM2.5 data
county_pm = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/county_pm25.csv")
county_temp = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/temp_seasonal_county.csv")

In [22]:
# Import census, brfss, testing, mortality, hosptial beds data as potential confounders
county_census = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/census_county_interpolated.csv")
county_brfss = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/brfss_county_interpolated.csv")
state_test = pd.read_csv("https://covidtracking.com/api/states/daily.csv")
state_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      2340 non-null   int64  
 1   state                     2340 non-null   object 
 2   positive                  2325 non-null   float64
 3   negative                  2184 non-null   float64
 4   pending                   552 non-null    float64
 5   hospitalizedCurrently     495 non-null    float64
 6   hospitalizedCumulative    713 non-null    float64
 7   inIcuCurrently            230 non-null    float64
 8   inIcuCumulative           118 non-null    float64
 9   onVentilatorCurrently     194 non-null    float64
 10  onVentilatorCumulative    36 non-null     float64
 11  recovered                 460 non-null    float64
 12  hash                      2340 non-null   object 
 13  dateChecked               2340 non-null   object 
 14  death   

In [90]:
date_clean = date_of_study.replace("-","",-1)
year = date_clean[4:8]
month_day = date_clean[0:4]
state_test = state_test[state_test['date'] == "{}{}".format('2020','0404')].drop(columns='fips')
#state_test = subset(state_test, date ==paste0(substring(str_remove_all(date_of_study, "-"),5,8),substring(str_remove_all(date_of_study, "-"),1,4)))[,-20]
statecode = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/statecode.csv")

/home/angelo/anaconda3/envs/flatiron/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


KeyError: "['fips'] not found in axis"

In [24]:
state_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      0 non-null      int64  
 1   state                     0 non-null      object 
 2   positive                  0 non-null      float64
 3   negative                  0 non-null      float64
 4   pending                   0 non-null      float64
 5   hospitalizedCurrently     0 non-null      float64
 6   hospitalizedCumulative    0 non-null      float64
 7   inIcuCurrently            0 non-null      float64
 8   inIcuCumulative           0 non-null      float64
 9   onVentilatorCurrently     0 non-null      float64
 10  onVentilatorCumulative    0 non-null      float64
 11  recovered                 0 non-null      float64
 12  hash                      0 non-null      object 
 13  dateChecked               0 non-null      object 
 14  death                 

In [89]:
hospitals = pd.read_csv("https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D")
hospitals['BEDS'][hospitals['BEDS'] < 0] = 0

/home/angelo/anaconda3/envs/flatiron/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
county_base_mortality = pd.read_table("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/county_base_mortality.txt", sep='\t')
county_old_mortality = pd.read_table("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/county_old_mortality.txt",  sep='\t')
county_old_mortality.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 3145 entries, nan to nan
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   County       3145 non-null   object 
 1   County Code  3145 non-null   int64  
 2   Deaths       3145 non-null   int64  
 3   Population   3145 non-null   int64  
 4   Crude Rate   3145 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 147.4+ KB


In [38]:
county_old_mortality=county_old_mortality.rename(columns={"Population": "older_Population"})
county_old_mortality.head()

,County,County Code,Deaths,older_Population,Crude Rate
NaN,"Autauga County, AL",1001,5338,109652,4868.1
NaN,"Baldwin County, AL",1003,22452,526184,4266.9
NaN,"Barbour County, AL",1005,3675,71542,5136.8
NaN,"Bibb County, AL",1007,2685,51438,5219.9
NaN,"Blount County, AL",1009,7076,146136,4842.1


In [39]:
county_base_mortality.head()

,County,County Code,Deaths,Population,Crude Rate_x,Age Adjusted Rate,% of Total Deaths,Crude Rate_y
0,"Autauga County, AL",1001,7893,918492,859.3,958.8,0.0%,4868.1
1,"Baldwin County, AL",1003,30292,3102984,976.2,812.6,0.1%,4266.9
2,"Barbour County, AL",1005,5197,499262,1040.9,951.7,0.0%,5136.8
3,"Bibb County, AL",1007,4089,397470,1028.8,1025.5,0.0%,5219.9
4,"Blount County, AL",1009,9912,997531,993.7,929.4,0.0%,4842.1


In [41]:
county_old_mortality1= county_old_mortality.iloc[:,[1,4]]

county_base_mortality = county_base_mortality.merge(county_old_mortality1,on="County Code", how='inner',)

county_base_mortality['older_pecent'] = county_base_mortality['Population']/county_base_mortality['Population']

In [42]:
statecode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   51 non-null     object
 1   Abbrev  51 non-null     object
 2   Code    51 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [43]:
# merging data
state_test = pd.merge(left=state_test, right=statecode, left_on='state', right_on='Code')

In [44]:
county_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54961 entries, 0 to 54960
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fips         54961 non-null  int64  
 1   year         54961 non-null  int64  
 2   summer_tmmx  52836 non-null  float64
 3   summer_rmax  52836 non-null  float64
 4   winter_tmmx  52836 non-null  float64
 5   winter_rmax  52836 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 2.5 MB


In [45]:
# pm average over 17 years



county_pm_aggregated = county_pm.groupby('fips').agg(mean_pm25=('pm25','mean')).reset_index()
# pm most recent 2016
#county_pm_aggregated = subset(county_pm , year==2016)
#county_pm_aggregated$mean_pm25 = county_pm_aggregated$pm25

# pm average over 17 years
county_temp_aggregated=county_temp.groupby('fips').agg(mean_winter_temp =  ('winter_tmmx', 'mean'), 
                                                       mean_summer_temp= ('summer_tmmx', 'mean'), 
                                                       mean_winter_rm= ('winter_rmax', 'mean'), 
                                                       mean_summer_rm= ('summer_rmax', 'mean')).reset_index()
                                                        


In [46]:
county_pm_aggregated = county_pm_aggregated.merge(county_temp_aggregated,on="fips",how='left')


In [47]:
county_hospitals_aggregated = hospitals.groupby('COUNTYFIPS').agg(beds =('BEDS', sum)).reset_index()
                                                        


In [ ]:
county_census_aggregated2 = county_census[county_census['year']==2016]
county_census_aggregated2['q_popdensity'] = 1
quantile_popdensity = county_census_aggregated2['popdensity'].quantile([0.25,0.5,0.75])



In [49]:
county_census_aggregated2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3208 entries, 17 to 57743
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fips                3207 non-null   float64
 1   year                3208 non-null   int64  
 2   poverty             3208 non-null   float64
 3   popdensity          3208 non-null   float64
 4   medianhousevalue    3208 non-null   float64
 5   pct_blk             3208 non-null   float64
 6   medhouseholdincome  3208 non-null   float64
 7   pct_owner_occ       3208 non-null   float64
 8   hispanic            3208 non-null   float64
 9   education           3208 non-null   float64
 10  population          3208 non-null   float64
 11  pct_asian           3208 non-null   float64
 12  pct_native          3208 non-null   float64
 13  pct_white           3208 non-null   float64
 14  q_popdensity        3208 non-null   int64  
dtypes: float64(13), int64(2)
memory usage: 401.0 KB


In [86]:


county_census_aggregated2.loc[county_census_aggregated2['popdensity'] <= quantile_popdensity[0.25], 'q_popdensity'] = 1



In [51]:
county_census_aggregated2.loc[(county_census_aggregated2['popdensity'] > quantile_popdensity[0.25]) &
                              (county_census_aggregated2['popdensity'] <= quantile_popdensity[0.50]), 'q_popdensity'] = 2




In [52]:
county_census_aggregated2.loc[(county_census_aggregated2['popdensity'] > quantile_popdensity[0.50]) & 
                              (county_census_aggregated2['popdensity'] <= quantile_popdensity[0.75]), 'q_popdensity'] = 3


county_census_aggregated2.loc[county_census_aggregated2['popdensity'] > quantile_popdensity[0.75], 'q_popdensity'] = 4




In [53]:
county_brfss_aggregated = county_brfss[county_brfss['year']==2012]

county_census_aggregated2 = county_census_aggregated2.merge(county_brfss_aggregated,on="fips",how='left')



In [54]:


aggregate_pm = pd.merge(left=county_pm_aggregated,right=covid_us,left_on="fips",right_on = "FIPS")

aggregate_pm_census = pd.merge(left=aggregate_pm,right=county_census_aggregated2,left_on="fips",right_on = "fips")







In [55]:
aggregate_pm_census_cdc = pd.merge(left=aggregate_pm_census,right=county_base_mortality.iloc[:,[1,4,5,8,9]],left_on = "fips",right_on = "County Code",how='left')

In [56]:
aggregate_pm_census_cdc = aggregate_pm_census_cdc[(aggregate_pm_census['fips'].isnull() == False)]

In [72]:
aggregate_pm_census_cdc_test.head()

,mean_pm25,mean_winter_temp,mean_summer_temp,mean_winter_rm,mean_summer_rm,FIPS,Admin2,Province_State,Country_Region,Last_Update,...,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,State,Abbrev,Code,population_frac_county


In [83]:
aggregate_pm_census_cdc_test = pd.merge(left=aggregate_pm_census_cdc,right=state_test,left_on="Province_State",right_on = "State")



In [87]:
aggregate_pm_census_cdc_test


total_pop = aggregate_pm_census_cdc_test['population'].sum()
aggregate_pm_census_cdc_test['population_frac_county']=[x['population']/total_pop for x in aggregate_pm_census_cdc_test.iterrows()]
aggregate_pm_census_cdc_test['totalTestResults_county']=[x['population_frac_county']*x['totalTestResults']for x in aggregate_pm_census_cdc_test.iterrows()]
                          

In [156]:
aggregate_pm_census_cdc_test_beds = pd.merge(left=aggregate_pm_census_cdc_test,right=county_hospitals_aggregated,left_on = "fips",right_on = "COUNTYFIPS",how = 'left')